#**pip installs**

In [ ]:
!pip install requests beautifulsoup4 nltk firebase


#**reset the DB**




In [ ]:
from firebase import firebase
FBconn = firebase.FirebaseApplication('https://beeteamcloud-default-rtdb.firebaseio.com/', None)
FBconn.delete('/', None)

#**SearchEngine**

In [ ]:
import requests , nltk , re , time , math
from bs4 import BeautifulSoup
from collections import defaultdict
from nltk.corpus import stopwords
from firebase import firebase
from nltk.stem import WordNetLemmatizer
from urllib.parse import urljoin

nltk.download('stopwords')
nltk.download('wordnet')
FBconn = firebase.FirebaseApplication('https://beeteamcloud-default-rtdb.firebaseio.com/', None)

class SearchEngine:
    def __init__(self):
        """Initialize the search engine."""
        self.base_url = "https://www.heroku.com"
        self.pages = []
        self.visited_pages = set()
        self.to_visit_pages = []
        self.word_locations = defaultdict(list)  # word -> [(page_id, frequency), ...]
        self.stop_words = set(stopwords.words('english'))
        self.idf = {}
        self.word_doc_count = defaultdict(int)  # word -> document count
        self.wnl = WordNetLemmatizer()

    def fetch_page(self,url):
        """Fetch a web page and return its parsed soup."""
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        else:
            return None

    def fetch_pages(self, url):
        """Fetch a web page and process its content."""
        if url in self.visited_pages:
            return

        soup = self.fetch_page(url)
        if soup is None:
            return

        self.visited_pages.add(url)

        try:
            title = soup.title.string.strip() if soup.title else "No Title"
        except AttributeError:
            title = "No Title"

        for link in soup.find_all('a', href=True):
            href = link.get('href')
            full_url = urljoin(self.base_url, href)  # Normalize the URL

            if full_url not in self.visited_pages:
                self.to_visit_pages.append(full_url)

        content = soup.get_text(separator = ' ', strip=True)
        page_id = len(self.pages) + 1
        self.pages.append({
            'id': page_id,
            'title': title,
            'url': url,
            'content': content
        })

        print(f"Retrieved: {title}")
        return True

    def scrape_all_website(self, max_pages = 25, delay = 0.01):
        """Perform web scraping on the Heroku website."""
        self.to_visit_pages.append(self.base_url)
        pages_visited = 0

        while self.to_visit_pages and pages_visited < max_pages:
            url = self.to_visit_pages.pop(0)
            if url in self.visited_pages:
                continue
            self.fetch_pages(url)
            pages_visited += 1
            time.sleep(delay)

        return self.pages

    def build_index(self):
        """Build a TF-IDF index."""
        self.word_locations.clear()
        self.word_doc_count.clear()
        self.idf = {}
        pages = self.scrape_all_website()
        total_pages = len(self.pages)

        # Calculate term frequencies (TF) and document counts
        for page in self.pages:
            words = re.findall(r'[a-z]{2,}', page['content'].lower())
            words = [self.wnl.lemmatize(word) for word in words]
            total_words = len(words)
            word_counts = defaultdict(int)

            # Count word occurrences in the current page
            for word in words:
                if word not in self.stop_words:
                    word_counts[word] += 1

            # Normalize term frequencies and store them
            for word, count in word_counts.items():
                tf = count / total_words
                self.word_locations[word].append((page['id'], page['url'], tf, count))
                self.word_doc_count[word] += 1

        # Calculate inverse document frequency (IDF)
        for word, doc_count in self.word_doc_count.items():
            self.idf[word] = math.log(total_pages / doc_count) if doc_count > 0 else 0

        # Process each word and store the top 5 documents (based on TF-IDF score)
        for word, locations in self.word_locations.items():
        # Calculate TF-IDF for each page containing the word
            word_scores = []
            for page_id, url, tf, count in locations:
                tfidf_score = tf * self.idf.get(word, 0)
                word_scores.append((page_id, url, tfidf_score, count))

            # Sort the scores in descending order to get the top 5 pages
            word_scores.sort(key=lambda x: x[2], reverse=True)
            top_pages = word_scores[:5]  # Keep top 5 pages

            # Extract the list of URLs (document IDs) where the word appears, limited to the top 5
            doc_urls = [page[1] for page in top_pages]

            # Prepare the document data for Firestore
            index_data = {
                'term': word,
                'DocIDs': doc_urls
            }

            # Save to Firebase Realtime Database
            FBconn.put('/indexs', word, index_data)

            print(f'index: {word} is built and saved to the database.')

    def search(self, query="", num_results = 5):
        """Search pages using TF-IDF ranking based on the stored index."""
        # Get query words, excluding stopwords
        query_words = [word.lower() for word in re.findall(r'\w+', query) if word.lower() not in self.stop_words]
        query_words = [self.wnl.lemmatize(word) for word in query_words]

        if not query_words:
            print("No valid query words provided.")
            return []

        # Retrieve the top pages for each word from the Firebase index
        page_scores = defaultdict(float)
        for word in query_words:
            try:
                result = FBconn.get('/indexs', word)
                if result:
                    doc_urls = result.get('DocIDs', [])
                    for url in doc_urls:
                        page_scores[url] += 1  # Increment score for each document where the word appears
            except Exception as e:
                    print(f"Failed to retrieve data for word '{word}': {e}")

        # Sort pages by their score (TF-IDF based ranking)
        ranked_results = sorted(page_scores.items(), key=lambda x: x[1], reverse=True)

        # Retrieve top results
        results = []
        for url, score in ranked_results[:num_results]:
            page = next((p for p in self.pages if p['url'] == url), None)
            if page:
                context = self.get_context(page['content'], query_words)
                results.append({
                    'title': page['title'],
                    'url': page['url'],
                    'context': context
                    })
        if not results:
            print("No results found for your query.")
        else:
            print(f"Top {num_results} search results found for '{query}'.")

        # Check if the query exists in the database, if not, save the results
        if not self.query_exists_in_db(query):
            self.save_search_result_to_db(query, results)

        return results

    def query_exists_in_db(self, query):
        """Check if the query already exists in the Firebase database."""
        try:
            # Use a hashed version of the query for checking
            result = FBconn.get(f'/search_results/{query}', None)
            return result is not None
        except Exception as e:
            print(f"Failed to check query in database: {e}")
            return False

    def save_search_result_to_db(self, query, results):
        """Save a search result to the Firebase Realtime Database."""
        try:
                # Save the search result under the 'search_results' node with the query as the key
            result_data = {
              'query': query,
              'results': results
            }
            FBconn.put('/search_results', query, result_data)
            print("Search result saved to database.")
        except Exception as e:
            print(f"Failed to save search result to database: {e}")

    def get_context(self, content, query_words, context_size = 100):
        """Get content snippet showing context of all query word matches."""
        content_lower = content.lower()
        matches = []

        for word in query_words:
            pos = content_lower.find(word)
            if pos != -1:
                matches.append(pos)

        if not matches:
            return content[:1000] + "..."

        # Expand the context around all matches
        start = max(0, min(matches) - context_size)
        end = min(len(content), max(matches) + context_size)

        return "..." + content[start:end] + "..."

    def init_search_engine(self):
        print("Starting to scrape the Heroku website...")
        print("Building the search index...")
        self.build_index()
        print("====================================================================================")
        print("init_search_engine is finished")
        print("====================================================================================")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **GUI**

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from firebase import firebase

engine = SearchEngine()
engine.init_search_engine()
results = []
# Create Tabs
tab1_content = widgets.Output()
with tab1_content:
    clear_output()
    FBconn = firebase.FirebaseApplication('https://beeteamcloud-default-rtdb.firebaseio.com/', None)

    # Elements of the interface
    term_input = widgets.Text(description = "Term:")
    url_input = widgets.Text(description = "URL:")
    add_button = widgets.Button(description = "Add Index")
    delete_input = widgets.Text(description = "Delete by Term:")
    delete_button = widgets.Button(description = "Delete Index")
    update_id_input = widgets.Text(description = "Update by ID:")
    update_term_input = widgets.Text(description = "New Term:")
    update_url_input = widgets.Text(description = "New URL:")
    update_button = widgets.Button(description = "Update Index")
    #refresh_button = widgets.Button(description = "Refresh List")
    output = widgets.Output()

    # Adding a new index
    def add_index(term):
        term = term_input.value
        url = url_input.value
        if term and url:
            upload = {'term': term, 'DocIds': url}
            FBconn.post('/indexs/', upload)
            with output:
                clear_output()
                print(f"Additional index: {term} ({url})")
        term_input.value = ""
        url_input.value = ""

    # Deleting an index by Term
    def delete_index_by_name(term):
        """Delete an index by term name from the Firebase database."""
        if not term:
              print("Term name is required for deletion.")
              return
        try:
            index_data = FBconn.get('/indexs', None)
            if not index_data:
                print("No indexes found in the database.")
                return
            # Search for the matching term
            deleted = False
            for index_id, data in index_data.items():
                if data.get('term') == term:
                    FBconn.delete('/indexs', index_id)
                    print(f"index deleted: {term}")
                deleted = True
                break

            if not deleted:
                  print(f"No index found with the term name: {term}")

        except Exception as e:
                print(f"An error occurred while deleting the index: {e}")

    # Update index by term
    def update_index(term):
        index_id = update_id_input.value
        new_term = update_term_input.value
        new_url = update_url_input.value

        if index_id and (new_term or new_url):
            data = FBconn.get(f'/indexs/{term}', None)
            if data:
                updated_data = {
                    'term': new_term if new_term else data['term'],
                    'DocIds': new_url if new_url else data['DocIds']
                    }
                FBconn.put('/indexs/',term, updated_data)
            with output:
                clear_output()
                print(f"index updated: {index_id}")
            update_id_input.value = ""
            update_term_input.value = ""
            update_url_input.value = ""

    # Link buttons to functions
    add_button.on_click(add_index)
    delete_button.on_click(delete_index_by_name)
    update_button.on_click(update_index)
    #refresh_button.on_click(refresh_tab5_action)

    # Showing the interface
    display(term_input, url_input, add_button)
    display(delete_input, delete_button)
    display(update_id_input, update_term_input, update_url_input, update_button)
    #display(refresh_button)

tab2_content = widgets.Output()
with tab2_content:
    clear_output()
    query_input = widgets.Text(description = "Query:")
    search_button = widgets.Button(description = "Search")

    # Action to handle the query and update results in Tab 3
    def search_query(_):
        query = query_input.value.strip()
        if query == "":
            return
        links = []
        # Perform the search with the engine
        results = engine.search(query)
        for result in results:
            links.append(result['url'])
        query_input.value = ""
        update_results(query, results)

    def refresh_tab5_action(_):
        display_all_search_history()

    search_button.on_click(search_query)
    search_button.on_click(refresh_tab5_action)
    display(query_input, search_button)

tab3_content = widgets.Output()
with tab3_content:
    clear_output()
    pass

def update_results(query, results):
    """This function will update Tab 3 with the results from the search."""
    with tab3_content:
        clear_output()
        if results:
            print(f"Results for: {query}")
            for result in results:
                print(f"- {result['title']} ({result['url']})")
        else:
            print(f"No results found for: {query}")


tab4_content = widgets.Output()
with tab4_content:
  clear_output()
  def visualize_term_data(word_doc_count, word_locations, top_n=50):
    """Visualize term frequency for the top N terms."""
    # Calculate term frequencies
    term_frequencies = {term: sum(count for _, _, _, count in word_locations[term]) for term in word_doc_count}

    # Sort terms by frequency in descending order
    sorted_terms = sorted(term_frequencies.items(), key=lambda x: x[1], reverse=True)[:top_n]

    terms = [term for term, _ in sorted_terms]
    frequencies = [freq for _, freq in sorted_terms]

    # Create the graph
    plt.figure(figsize=(10, 6))
    plt.bar(terms, frequencies, color='blue', alpha=0.8, label='Term Frequency')
    plt.title(f"Top {top_n} Terms by Frequency")
    plt.xlabel("Terms")
    plt.ylabel("Frequency")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.legend()
    plt.show()


  def visualize_page_lengths(pages, top_n = 50):
    """Visualize the top N pages by content length."""
    # Sort pages by content length
    sorted_pages = sorted(pages, key=lambda x: len(x['content']), reverse=True)[:top_n]
    titles = [page['title'] for page in sorted_pages]
    content_lengths = [len(page['content']) for page in sorted_pages]

    # Create the graph
    plt.figure(figsize=(10, 6))
    plt.barh(titles, content_lengths, color='green')
    plt.title(f"Top {top_n} Pages by Content Length")
    plt.xlabel("Content Length (Characters)")
    plt.ylabel("Page Titles")
    plt.gca().invert_yaxis()  # Reverse the order for better readability
    plt.tight_layout()
    plt.show()

  pages = engine.pages
  word_doc_count = engine.word_doc_count
  word_locations = engine.word_locations
  # Visualize term data
  visualize_term_data(word_doc_count, word_locations)
  # Visualize page lengths
  visualize_page_lengths(pages)

def display_all_search_history():
    search_history = FBconn.get('/search_results/', None)  # Fetch all search results from Firebase
    with tab5_content:
            clear_output()
            if search_history:
                print("Search History:")
                for idx, data in search_history.items():
                    query = data.get('query', 'Unknown Query')
                    results = data.get('results', [])
                    print(f"Query: {query}")
                    print("Results:")
                    for result in results:
                        title = result.get('title', 'No Title')
                        url = result.get('url', 'No URL')
                        print(f"  - Title: {title}")
                        print(f"    URL: {url}")
                    print("-" * 50)  # Separator for readability
            else:
                print("No search history to display.")

tab5_content = widgets.Output()
with tab5_content:
    clear_output()
    display_all_search_history()


tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content, tab5_content])

tabs.set_title(0, 'Tab 1: admim options')
tabs.set_title(1, 'Tab 2: query')
tabs.set_title(2, 'Tab 3: query results')
tabs.set_title(3, 'Tab 4: Charts and stas')
tabs.set_title(4, 'Tab 5: searchs history from the DB')

# Display Tabs
clear_output()
display(tabs)